# Generate book table of contents

Here we generate the Jupyter book table of contents file at `docs/_toc.yml`.

In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.
contigs = ["3L"]
dask_scheduler = "threads"
analysis_version = "2025.01.30"

In [ ]:
import pandas as pd
import geopandas as gpd
from pyprojroot import here
import yaml
import dask

dask.config.set(scheduler=dask_scheduler);

In [ ]:
gdf_cohorts = gpd.read_file(
    here() / "build" / analysis_version / "final_cohorts.geojson"
)
gdf_cohorts.head()

In [ ]:
gdf_cohorts.columns

In [ ]:
countries = gdf_cohorts["country_alpha2"].unique()
countries

In [ ]:
# discover alerts
alert_dir = here() / "workflow" / "alerts"
alerts = sorted([
    f.name.split(".")[0] 
    for f in alert_dir.iterdir()
    if f.name.startswith("SA-")]
)
alerts

In [ ]:
alerts_part = dict(
    caption="Alerts",
    chapters=[
        dict(
            file="alerts",
            sections=[dict(file=f"alert/{a}") for a in alerts],
        ),
    ],
)
alerts_part

In [ ]:
genome_part = dict(
    caption="By chromosome",
    chapters=[
        dict(
            file=f"genome/ag-{contig}",
        )
        for contig in contigs
    ],
)
genome_part

In [ ]:
def build_country_chapter(country):
    cohorts = sorted(gdf_cohorts.query(f"country_alpha2 == '{country}'")["cohort_id"])
    chapter = dict(
        file=f"country/{country}",
        sections=[
            dict(
                file=f"cohort/{cohort}",
            )
            for cohort in cohorts
        ],
    )
    return chapter


In [ ]:
countries_part = dict(
    caption="By country",
    chapters=[build_country_chapter(country) for country in countries],
)
countries_part

In [ ]:
toc = dict(
    format="jb-book",
    root="home-page",
    parts=[
        alerts_part,
        genome_part,
        countries_part,
    ],
)
toc

In [ ]:
with open(here() / "docs" / "_toc.yml", mode="w") as output_file:
    yaml.safe_dump(toc, output_file)